# Decision Tree Regression Spark

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Set up the environment for using pyspark
import findspark

findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.linalg import Vectors

In [ ]:
# Create Application Context
spark = SparkSession.builder.appName("Decision Tree Example").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("Error")

## Load Data

In [ ]:
# Read the dataset file which is in csv - comma separated values format
sdf = spark.read.format('csv').options(header='true', inferSchema='true').load('../datasets/Position_Salaries.csv')

In [ ]:
sdf.show()

In [ ]:
from pyspark.ml.feature import VectorAssembler
vassemb = VectorAssembler(inputCols = ['Level'], outputCol = 'features')
ndf = vassemb.transform(sdf)
ndf = ndf.select(['features', 'Salary'])
ndf.show(3)

In [ ]:
ndf.printSchema()

## Decision Tree Regressor

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
(trainingData, testData) = ndf.randomSplit([0.7, 0.3], seed = 2345)

In [ ]:
# Create an initial Decision Tree Model
dt = DecisionTreeRegressor(labelCol="Salary", featuresCol="features")

In [ ]:
dt_model = dt.fit(trainingData)

In [ ]:
dt_predictions = dt_model.transform(testData)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
dt_evaluator = RegressionEvaluator(
    labelCol="Salary", predictionCol="prediction", metricName="rmse")

In [ ]:
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [ ]:
lr_predictions = dt_model.transform(testData)
lr_predictions.select("prediction","Salary","features").show(5)